In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.metrics import get_scorer_names
import optuna 
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import learning_curve

#############################
import lightgbm as lgb
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from scipy import stats
from statsmodels.stats.outliers_influence import variance_inflation_factor
#################################
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest, chi2
import warnings
warnings.filterwarnings('ignore')
import missingno as msno

from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance
from sklearn.impute import KNNImputer

In [ ]:
trainx = pd.read_csv('../input/playground-series-s3e22/train.csv')
test = pd.read_csv('../input/playground-series-s3e22/test.csv')
original = pd.read_csv('../input/horse-survival-dataset/horse.csv')
submission = pd.read_csv('../input/playground-series-s3e22/sample_submission.csv')

In [ ]:
train = pd.concat([trainx, original], ignore_index=True)
train.drop_duplicates(inplace=True)
total = pd.concat([train, test], ignore_index=True)
target='outcome'

In [ ]:
print('The dimension of the total dataset is:', total.shape)
print('The dimension of the test dataset is:', test.shape)
print('The dimension of the original train dataset is:', original.shape)
print('The dimension of the train dataset is:', trainx.shape)

In [ ]:
filtered_data = train[(train['peripheral_pulse'] =='reduced') ]


print(filtered_data['outcome'])
plt.figure(figsize=(8, 6))  
sns.countplot(data=filtered_data, x='outcome')


plt.xlabel('Outcome')
plt.ylabel('Count')
plt.title('Count Plot of Outcome for Filtered Data')


plt.show()

In [ ]:
total.drop('id',axis=1,inplace=True)


In [ ]:
sns.countplot(data = total, x = 'outcome')
plt.ylabel('Frequency');

In [ ]:
categorical_feats=[]
for column in total.columns:
  
    if total[column].dtype == 'object':
        
        categorical_feats.append(column)


print(categorical_feats)
categorical_feats.remove('outcome')

In [ ]:
numerical_features = []


for column in total.columns:
    
    if pd.api.types.is_numeric_dtype(total[column]):
     
        numerical_features.append(column)


print(numerical_features)

In [ ]:
total["outcome"] = total["outcome"].map({'euthanized':0,'died':1,'lived':2 })

In [ ]:
pd.options.display.float_format = '{:,.2f}'.format
def summary(df):
    print(f'data shape: {df.shape}')
    summ = pd.DataFrame(df.dtypes, columns=['data type'])
    summ['#missing'] = df.isnull().sum().values 
    summ['%missing'] = df.isnull().sum().values / len(df) * 100
    summ['#unique'] = df.nunique().values
    desc = pd.DataFrame(df.describe(include='all').transpose())
    summ['min'] = desc['min'].values
    summ['max'] = desc['max'].values
    summ['average'] = desc['mean'].values
    summ['standard_deviation'] = desc['std'].values
    summ['first value'] = df.loc[0].values
    summ['second value'] = df.loc[1].values
    summ['third value'] = df.loc[2].values
    
    return summ

In [ ]:
summary(total).style.background_gradient(cmap='YlOrBr')


In [ ]:
plt.figure(figsize=(14, len(categorical_feats)*2.5))

for idx, column in enumerate(categorical_feats):
    plt.subplot(len(categorical_feats)//2 + len(categorical_feats) % 2, 2, idx+1)
    ax = sns.countplot(x=column, data=train, hue='outcome', palette='YlOrRd')
    plt.title(f"{column} Countplot by outcome")
    plt.ylim(0, train[column].value_counts().max() + 10)

    for p in ax.patches:
        height = p.get_height()
        plt.text(p.get_x() + p.get_width()/2., height + 2, height, ha="center")

    # Remove duplicate legend
    if idx != 0:
        ax.get_legend().remove()

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(14, len(categorical_feats)*2.5))

for idx, column in enumerate(categorical_feats):
    plt.subplot(len(categorical_feats)//2 + len(categorical_feats) % 2, 2, idx+1)
    sns.countplot(x=column, data=test, palette='YlOrRd')
    plt.title(f"{column} Countplot")

    plt.ylim(0, test[column].value_counts().max() + 10)

    # Add count labels above each rectangular bar
    for p in sns.countplot(x=column, data=test, palette='YlOrRd').patches:
        height = p.get_height()
        plt.text(p.get_x() + p.get_width()/2., height + 2, height, ha="center")

plt.tight_layout()
plt.show()



In [ ]:
total["rectal_exam_feces"] = np.where(total["rectal_exam_feces"].isin(["serosanguious"]), total["rectal_exam_feces"].mode()[0], 
                                              total["rectal_exam_feces"])
total["pain"] = np.where(total["pain"].isin(["slight"]), "mild_pain", total["pain"])
total["peristalsis"] = np.where(total["peristalsis"].isin(["distend_small"]), total["peristalsis"].mode()[0], 
                                              total["peristalsis"])

total["nasogastric_reflux"] = np.where(total["nasogastric_reflux"].isin(["slight"]), 
                                               total["nasogastric_reflux"].mode()[0],
                                               total["nasogastric_reflux"])


In [ ]:
total['frequency'] = total['hospital_number'].map(total['hospital_number'].value_counts())

In [ ]:
total.drop('hospital_number' , axis =1 , inplace = True )

In [ ]:
total['frequency']

In [ ]:
sns.pairplot(data=train[numerical_features+[target]], hue=target , corner=True)

In [ ]:
fig, axes = plt.subplots(nrows=6, ncols=2, figsize=(12, 16))

for i, feature in enumerate(numerical_features):
    row = i // 2
    col = i % 2
    sns.boxplot(ax=axes[row, col], data=train, x='outcome', y=feature)
    axes[row, col].set_title(f'Boxplot of {feature}')
    axes[row, col].set_xlabel('Outcome')
    axes[row, col].set_ylabel(feature)

# Remove empty subplots if the number of features is not divisible by 2
if len(numerical_features) % 2 != 0:
    axes[-1, -1].remove()

plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(nrows=6, ncols=2, figsize=(12, 16))

for i, feature in enumerate(numerical_features):
    row = i // 2
    col = i % 2
    sns.boxplot(ax=axes[row, col], data=test,  y=feature)
    axes[row, col].set_title(f'Boxplot of {feature}')
    axes[row, col].set_xlabel('Outcome')
    axes[row, col].set_ylabel(feature)

# Remove empty subplots if the number of features is not divisible by 2
if len(numerical_features) % 2 != 0:
    axes[-1, -1].remove()

plt.tight_layout()
plt.show()

In [ ]:
plt.scatter(train['packed_cell_volume'], train['outcome'], alpha=0.5)
plt.xlabel('packed_cell_volume')
plt.ylabel('outcome')
plt.title('Scatter Plot: PCV vs. Target Variable')
plt.show()

In [ ]:
plt.scatter(train['total_protein'], train['outcome'], alpha=0.5)
plt.xlabel('total_protein')
plt.ylabel('outcome')
plt.title('Scatter Plot: TP vs. Target Variable')
plt.show()

In [ ]:
correlation_matrix = total[['packed_cell_volume', 'total_protein', 'outcome']].corr()

In [ ]:
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Heatmap')
plt.show()

In [ ]:
def IQR(col):
    q1 = np.percentile(col, 25)
    q3 = np.percentile(col, 75)
    iqr = q3 - q1
    upper_bound = q3 + (1.5 * iqr)
    upper_outliers = col > upper_bound

    # Replace outliers with upper bound values
    col = np.where(upper_outliers, round(upper_bound, 2), col)  
    return col


total["total_protein"] = IQR(total["total_protein"])

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(x='outcome', y='rectal_temp', data=train)
plt.title('Box Plot of Rectal Temperature vs Outcome')
plt.xlabel('Outcome')
plt.ylabel('Rectal Temperature')
plt.show()

# Create a violin plot
plt.figure(figsize=(10, 6))
sns.violinplot(x='outcome', y='rectal_temp', data=total)
plt.title('Violin Plot of Rectal Temperature vs Outcome')
plt.xlabel('Outcome')
plt.ylabel('Rectal Temperature')
plt.show()

In [ ]:
total['lesion_1'] = total['lesion_1'].astype(str)
site_of_lesion_mapping = {
    '1': 'gastric',
    '2': 'small intestine',
    '3': 'large colon',
    '4': 'large colon and cecum',
    '5': 'cecum',
    '6': 'transverse colon',
    '7': 'rectum/descending colon',
    '8': 'uterus',
    '9': 'bladder',
    '11': 'all intestinal sites',
    '0': 'none',
}

type_mapping = {
    '1': 'simple',
    '2': 'strangulation',
    '3': 'inflammation',
    '4': 'other',
}

subtype_mapping = {
    '1': 'mechanical',
    '2': 'paralytic',
    '0': 'n/a',
}

specific_code_mapping = {
    '1': 'obturation',
    '2': 'intrinsic',
    '3': 'extrinsic',
    '4': 'adynamic',
    '5': 'volvulus/torsion',
    '6': 'intussusception',
    '7': 'thromboembolic',
    '8': 'hernia',
    '9': 'lipoma/splenic incarceration',
    '10': 'displacement',
    '0': 'n/a',
}


total['site'] = total['lesion_1'].str[0].map(site_of_lesion_mapping)
total['type'] = total['lesion_1'].str[1].map(type_mapping)
total['subtype'] = total['lesion_1'].str[2].map(subtype_mapping)
total['specific_code'] = total['lesion_1'].str[3].map(specific_code_mapping)
special_cases = {
    '31110': ('large colon', 'simple', 'mechanical', 'displacement'),
    '12208': ('small intestine', 'strangulation', 'n/a', 'hernia'),
    '11300': ('gastric', 'simple', 'n/a', 'n/a'),
    '11400': ('all intestinal sites', 'other', 'n/a', 'n/a'),
    '11124': ('all intestinal sites', 'simple', 'paralytic', 'adynamic'),
    '21110': ('small intestine', 'simple', 'mechanical', 'displacement'),
    '41110': ('large colon and cecum', 'simple', 'mechanical', 'displacement'),
}


for case, (site, type, subtype, specific_code) in special_cases.items():
    total.loc[total['lesion_1'] == case, 'site'] = site
    total.loc[total['lesion_1'] == case, 'type'] = type
    total.loc[total['lesion_1'] == case, 'subtype'] = subtype
    total.loc[total['lesion_1'] == case, 'specific_code'] = specific_code

total.drop(['lesion_1', 'lesion_2', 'lesion_3'], axis=1, inplace=True)

In [ ]:
total['site'].value_counts()

In [ ]:
categorical_feats = []
for column in total.columns:
   
    if total[column].dtype == 'object':
    
        categorical_feats.append(column)


print(categorical_feats)

In [ ]:
numerical_features = []


for column in total.columns:
    
    if pd.api.types.is_numeric_dtype(total[column]):
     
        numerical_features.append(column)


print(numerical_features)

In [ ]:
def plot_correlation_matrix(data, numerical_features):
    # Select the numerical features
    numerical_data = data[numerical_features]

    # Compute the correlation matrix
    correlation_matrix = numerical_data.corr()

    # Generate a mask for the upper triangle to hide redundant values
    mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))

    # Set up the matplotlib figure
    fig, ax = plt.subplots(figsize=(10, 8))

    # Create a custom colormap
    cmap = sns.diverging_palette(230, 20, as_cmap=True)

    # Plot the correlation matrix
    sns.heatmap(correlation_matrix, mask=mask, cmap=cmap, center=0, annot=True,
                square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})

    # Customize the plot
    plt.title("Correlation Matrix of Numerical Features")
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()

    # Show the plot
    plt.show()

In [ ]:
plot_correlation_matrix(total, numerical_features)


In [ ]:
numerical_features.remove('outcome')

In [ ]:
for col in categorical_feats:
    total[col] = total[col].fillna(total[col].mode()[0])

imputer = KNNImputer(n_neighbors=8)  
total[numerical_features] = imputer.fit_transform(total[numerical_features])


In [ ]:
total['mucous_membrane'].unique()

In [ ]:
pulse_threshold = 80  
mucous_membrane_colors = ['pale_cyanotic', 'dark_cyanotic']
capillary_refill_criteria = 'more_3_sec'
pain_levels = ['severe_pain']
respiratory_rate_threshold = 25  
temperature_of_extremities_criteria = ['cold']  
perpulse = ['absent']
abdominal_distention = ['severe']
rectal = 36


pulse_weight = 0.3
mucous_membrane_weight = 0.25
capillary_refill_weight = 0.15
pain_weight = 0.2
respiratory_rate_weight = 0.15  
temperature_of_extremities_weight = 0.15 
peripheral_pulse_weight = 0.15
abdominal_distention_weight = 0.15
rectal_weight = 0.05



def calculate_shock_probability(row):
    
    probability = 0  

   
    if row['pulse'] > pulse_threshold:
        probability += pulse_weight
    if row['mucous_membrane'] in mucous_membrane_colors:
        probability += mucous_membrane_weight
    if row['capillary_refill_time'] == capillary_refill_criteria:
        probability += capillary_refill_weight
    if row['pain'] in pain_levels:
        probability += pain_weight
    if row['respiratory_rate'] > respiratory_rate_threshold:
        probability += respiratory_rate_weight
    if row['temp_of_extremities'] == temperature_of_extremities_criteria:
        probability += temperature_of_extremities_weight
        
    if row['peripheral_pulse'] == perpulse:
        probability += peripheral_pulse_weight   
    if row['abdominal_distention'] == abdominal_distention:
        probability += abdominal_distention_weight      

    if row['rectal_temp'] < rectal:
        probability += rectal_weight   
       
  
    probability = min(max(probability, 0), 1)

    return probability


total['shock_probability'] = total.apply(calculate_shock_probability, axis=1)


print(total['shock_probability'].describe())

In [ ]:
total["rectal_temp"] = (total["rectal_temp"] - 37.8).abs()


In [ ]:
encodings = {
    ('none', 'clear'): 11,
    ('slight', 'clear'): 10,
    ('none', 'cloudy'): 9,
    ('slight', 'cloudy'): 8,
    ('moderate', 'clear'): 7,
    ('moderate', 'cloudy'): 6,
    ('severe', 'clear'): 5,
    ('severe', 'cloudy'): 4,
    ('none', 'serosanguious'): 3,
    ('slight', 'serosanguious'): 2,
    ('moderate', 'serosanguious'): 1,
    ('severe', 'serosanguious'): 0
}

# Create the new encoded feature using the combinations
total['encoded_combination'] = total.apply(lambda row: encodings[(row['abdominal_distention'], row['abdomo_appearance'])], axis=1)

# Print the updated DataFrame to see the new feature
print(total['encoded_combination'].value_counts())

In [ ]:
total.drop(['abdominal_distention', 'abdomo_appearance'], axis=1, inplace=True)

In [ ]:
total["temp_of_extremities"] = total["temp_of_extremities"].map({'cool':0,'cold':1,'warm':2,'normal':3 })
total["peripheral_pulse"] = total["peripheral_pulse"].map({'absent':0,'reduced':1,'increased':2,'normal':3})
total["capillary_refill_time"] = total["capillary_refill_time"].map({'more_3_sec':0,'3':1,'less_3_sec':2})
total["pain"] = total["pain"].map({'severe_pain':0.0,'extreme_pain':1.0,'depressed':2.0,'moderate':3.0,'mild_pain':4.0,'alert':5.0})
total["peristalsis"] = total["peristalsis"].map({'absent':0,'hypermotile':1,'hypomotile':2,'normal':3})

total["nasogastric_reflux"] = total["nasogastric_reflux"].map({'more_1_liter':0,'less_1_liter':1,'none':2})
total["rectal_exam_feces"] = total["rectal_exam_feces"].map({"absent":0,'decreased':1,'increased':2,'normal':3})
total["nasogastric_tube"] = total["nasogastric_tube"].map({'significant':0,'slight':1,'none':2})
# total["abdominal_distention"] = total["abdominal_distention"].map({'none':0,'slight':1,'moderate':2,'severe':3})
# total["abdomo_appearance"] = total["abdomo_appearance"].map({'serosanguious':0,'cloudy':1,'clear':2})
total["abdomen"] = total["abdomen"].map({'distend_large':0,'distend_small':1,'firm':2,'other':3,'normal':4})
total["site"] = total["site"].map({'gastric':0,'small intestine':1,'large colon':2,'large colon and cecum':4,'cecum':3 , 'transverse colon':5,'rectum/descending colon':6,'uterus':7, 'bladder':8 ,'all intestinal sites':10 ,'none':9 })
total["type"] = total["type"].map({'simple':3,'inflammation':2,'strangulation':1,'other':0})
# total["subtype"] = total["subtype"].map({'paralytic':0,'mechanical':1,'n/a':2})
total["specific_code"] = total["specific_code"].map({'lipoma/splenic incarceration':0,'hernia':1,'adynamic':2,'volvulus/torsion':4,'obturation':3 , 'thromboembolic':5,'intussusception':6,'extrinsic':7, 'intrinsic':8 ,'n/a':9 ,'displacement':10 })
# total["mucous_membrane"] = total["mucous_membrane"].map({'dark_cyanotic' : 0 , 'pale_cyanotic' : 1 , 'bright_red':2 , 'pale_pink' : 4 , 'bright_pink': 3 , 'normal_pink' : 5})
total["surgery"] = total["surgery"].map({'yes':0,'no':1})
total["age"] = total["age"].map({'young':0,'adult':1})
total["surgical_lesion"] = total["surgical_lesion"].map({'yes':0,'no':1})
total["cp_data"] = total["cp_data"].map({'no':0,'yes':1})

In [ ]:
categorical_feats = []
for column in total.columns:
   
    if total[column].dtype == 'object':
       
        categorical_feats.append(column)


print(categorical_feats)

In [ ]:
numerical_features = []


for column in total.columns:
    
    if pd.api.types.is_numeric_dtype(total[column]):
     
        numerical_features.append(column)


print(numerical_features)


In [ ]:
encfeats=['surgery','age','surgical_lesion','cp_data','mucous_membrane']


In [ ]:
total_ohe = pd.get_dummies(total, columns=encfeats)

In [ ]:
total_ohe.columns

In [ ]:
total_ohe.drop(['cp_data_0', 'age_0', 'surgical_lesion_0','surgery_0', 'subtype','mucous_membrane_bright_pink'], axis=1, inplace=True)

In [ ]:
total_ohe.drop(['respiratory_rate' , 'abdomo_protein' , 'temp_of_extremities'], axis=1, inplace=True)

In [ ]:
total.isna().sum()

In [ ]:
train_l = total_ohe[total_ohe[target].notnull()]
test_l = total_ohe[total_ohe[target].isnull()].drop(columns=[target])


In [ ]:
X_train = train_l.drop(columns=['outcome'])
y_train = train_l['outcome']  

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(test_l)

In [ ]:
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=test_l.columns)

In [ ]:
X = X_train_scaled
y = y_train.astype(int)

In [ ]:
print(X.shape, y.shape)
y.value_counts()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV



# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Create a decision tree classifier
clf = DecisionTreeClassifier()

# Define a grid of hyperparameters to search over
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20, 30],  # You can adjust the depth values
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None],  # You can adjust this based on the number of features
    'random_state': [42]
}

# Perform grid search with cross-validation to find the best hyperparameters
grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Get the best hyperparameters from the grid search
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

# Create a decision tree classifier with the best hyperparameters
best_clf = DecisionTreeClassifier(**best_params)

# Fit the classifier to the training data
best_clf.fit(X_train, y_train)

# Make predictions on the test data
y_pred = best_clf.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Create a LightGBM dataset
train_data = lgb.Dataset(X_train, label=y_train)

# Define hyperparameters for tuning
param_grid = {
    'objective': ['multiclass'],
    'num_class': [3],
    'boosting_type': ['gbdt'],
    'metric': ['multi_logloss'],
    'num_leaves': [31, 50, 100],   
    'max_depth': [-1],             
    'learning_rate': [0.05, 0.1],  
    'feature_fraction': [1.0],     
    'bagging_fraction': [0.8],    
    'bagging_freq': [5],
    'verbose': [-1],
    'seed': [42],
}


clf = lgb.LGBMClassifier()

# Perform grid search with cross-validation to find the best hyperparameters
grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Get the best hyperparameters from the grid search
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

# Create a LightGBM classifier with the best hyperparameters
best_clf = lgb.LGBMClassifier(**best_params)

# Train the classifier on the training data
best_clf.fit(X_train, y_train)

# Make predictions on the test data
y_pred = best_clf.predict(X_test)

# Calculate accuracy and ROC AUC
accuracy = accuracy_score(y_test, y_pred)
y_prob = best_clf.predict_proba(X_test)
roc_auc = roc_auc_score(y_test, y_prob, multi_class='ovr')

print(f"Accuracy: {accuracy}")
print(f"ROC AUC: {roc_auc}")

# Plot the learning curve
def plot_learning_curve(estimator, X, y):
    train_sizes, train_scores, valid_scores = learning_curve(
        estimator, X, y, train_sizes=np.linspace(0.1, 1.0, 10), cv=5, scoring='accuracy'
    )

    train_mean = np.mean(train_scores, axis=1)
    train_std = np.std(train_scores, axis=1)
    valid_mean = np.mean(valid_scores, axis=1)
    valid_std = np.std(valid_scores, axis=1)

    plt.figure(figsize=(10, 6))
    plt.plot(train_sizes, train_mean, marker='o', label='Training Accuracy', color='b')
    plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.15, color='b')
    plt.plot(train_sizes, valid_mean, marker='o', label='Validation Accuracy', color='g')
    plt.fill_between(train_sizes, valid_mean - valid_std, valid_mean + valid_std, alpha=0.15, color='g')

    plt.title('Learning Curve')
    plt.xlabel('Training Examples')
    plt.ylabel('Accuracy')
    plt.legend(loc='best')
    plt.grid()
    plt.show()

# Plot the learning curve for the best LightGBM classifier
plot_learning_curve(best_clf, X_train, y_train)



In [ ]:
def plot_learning_curve(estimator, X, y):
    train_sizes, train_scores, valid_scores = learning_curve(
        estimator, X, y, train_sizes=np.linspace(0.1, 1.0, 10), cv=5, scoring='accuracy'
    )

    train_mean = np.mean(train_scores, axis=1)
    train_std = np.std(train_scores, axis=1)
    valid_mean = np.mean(valid_scores, axis=1)
    valid_std = np.std(valid_scores, axis=1)

    plt.figure(figsize=(10, 6))
    plt.plot(train_sizes, train_mean, marker='o', label='Training Accuracy', color='b')
    plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.15, color='b')
    plt.plot(train_sizes, valid_mean, marker='o', label='Validation Accuracy', color='g')
    plt.fill_between(train_sizes, valid_mean - valid_std, valid_mean + valid_std, alpha=0.15, color='g')

    plt.title('Learning Curve')
    plt.xlabel('Training Examples')
    plt.ylabel('Accuracy')
    plt.legend(loc='best')
    plt.grid()
    plt.show()

In [ ]:
def load_data():

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    return X_train, X_test, y_train, y_test

In [ ]:
def objective(trial):
    # hyperparameters to tune
    params = {
        'objective': 'multiclass',
        'num_class': 3,
        'boosting_type': 'gbdt',
        'metric': 'multi_logloss',
        'num_leaves': trial.suggest_int('num_leaves', 31, 100),
        'max_depth': -1,
        'learning_rate': trial.suggest_float('learning_rate', 0.05, 0.1),
        'feature_fraction': 1.0,
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
        'bagging_freq': 5,
        'verbose': -1,
        'seed': 42,
    }

    # Create a LightGBM classifier with the trial parameters
    clf = lgb.LGBMClassifier(**params)

    # Load the dataset
    X_train, X_test, y_train, y_test = load_data()

    # Train the classifier on the training data
    clf.fit(X_train, y_train)

    # Make predictions on the test data
    y_pred = clf.predict(X_test)

    # Calculate accuracy (you can replace this with your own metric)
    accuracy = accuracy_score(y_test, y_pred)

    return accuracy

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)  # You can adjust the number of trials

#the best hyperparameters
best_params = study.best_params
best_accuracy = study.best_value
print("Best Hyperparameters:", best_params)
print("Best Accuracy:", best_accuracy)

In [ ]:
#  LightGBM classifier with the best hyperparameters
best_clf = lgb.LGBMClassifier(**best_params)


X_train, X_test, y_train, y_test = load_data()

# Train classifier on the training data
best_clf.fit(X_train, y_train)

# Make predictions on the test data
y_pred = best_clf.predict(X_test)

# Calculate accuracy 
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Plot the learning curve
plot_learning_curve(best_clf, X_train, y_train)

In [ ]:
feature_importances = best_clf.feature_importances_

#  store and display feature importances
importance_df = pd.DataFrame({'Feature': X_train.columns, 'Importance': feature_importances})

# Sort by importance in descending order
importance_df = importance_df.sort_values(by='Importance', ascending=False)

In [ ]:
importance_df

In [ ]:
perm_importance = permutation_importance(best_clf, X_test, y_test, n_repeats=30, random_state=42)

# Visualize the feature importances
sorted_idx = perm_importance.importances_mean.argsort()[::-1]
plt.figure(figsize=(10, 6))
plt.bar(range(X_test.shape[1]), perm_importance.importances_mean[sorted_idx])
plt.xticks(range(X_test.shape[1]), sorted_idx)
plt.xlabel("Feature Index")
plt.ylabel("Permutation Importance")
plt.title("Permutation Feature Importance")
plt.show()

In [ ]:
best_clf.fit(X ,y)
trialll = best_clf.predict(X_test_scaled)

In [ ]:
final = best_clf.predict(X_test_scaled)

In [ ]:
submission = pd.DataFrame({'id': test['id'], 'outcome': trialll})
submission['outcome'] = submission['outcome'].map({1:'died',0:'euthanized',2:'lived'})
submission.to_csv('submission.csv',index=False)